In [7]:
import pandas as pd

#xls = pd.ExcelFile("LCDataDictionary.xlsx")
#df_dict_all = pd.concat([xls.parse(sheet) for sheet in xls.sheet_names], ignore_index=True)

In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan, countDistinct

In [11]:
spark = SparkSession.builder.getOrCreate()
df_loans = spark.read.csv("LCloan_raw_data.csv", header=True, inferSchema=True)

## Mapped file: 
### df_all.csv - combination of data dict sheets with their dtypes

### The goal in this doc: Mapping the relevant columns for testing -required

In [17]:
df_all =pd.read_csv('mapping_updated.csv')

In [19]:
df_all

,column_name,Description,source_sheet,type_status,column_name_fixed,column_type
0,amount requested,The total amount requested by the borrower,RejectStats,missing,amount requested,NaN
1,application date,The date which the borrower applied,RejectStats,missing,application date,NaN
2,loan title,The loan title provided by the borrower,RejectStats,missing,loan title,NaN
3,risk_score,"For applications prior to November 5, 2013 the...",RejectStats,missing,risk_score,NaN
4,debt-to-income ratio,A ratio calculated using the borrower’s total ...,RejectStats,missing,debt-to-income ratio,NaN
...,...,...,...,...,...,...
275,sec_app_num_rev_accts,Number of revolving accounts at time of appli...,BrowseNotes,found,sec_app_num_rev_accts,int
276,sec_app_chargeoff_within_12_mths,Number of charge-offs within last 12 months a...,BrowseNotes,found,sec_app_chargeoff_within_12_mths,int
277,sec_app_collections_12_mths_ex_med,Number of collections within last 12 months e...,BrowseNotes,found,sec_app_collections_12_mths_ex_med,int
278,sec_app_mths_since_last_major_derog,Months since most recent 90-day or worse rati...,BrowseNotes,found,sec_app_mths_since_last_major_derog,int


In [21]:
df_all['is_required'] = True

In [23]:
df_all

,column_name,Description,source_sheet,type_status,column_name_fixed,column_type,is_required
0,amount requested,The total amount requested by the borrower,RejectStats,missing,amount requested,NaN,True
1,application date,The date which the borrower applied,RejectStats,missing,application date,NaN,True
2,loan title,The loan title provided by the borrower,RejectStats,missing,loan title,NaN,True
3,risk_score,"For applications prior to November 5, 2013 the...",RejectStats,missing,risk_score,NaN,True
4,debt-to-income ratio,A ratio calculated using the borrower’s total ...,RejectStats,missing,debt-to-income ratio,NaN,True
...,...,...,...,...,...,...,...
275,sec_app_num_rev_accts,Number of revolving accounts at time of appli...,BrowseNotes,found,sec_app_num_rev_accts,int,True
276,sec_app_chargeoff_within_12_mths,Number of charge-offs within last 12 months a...,BrowseNotes,found,sec_app_chargeoff_within_12_mths,int,True
277,sec_app_collections_12_mths_ex_med,Number of collections within last 12 months e...,BrowseNotes,found,sec_app_collections_12_mths_ex_med,int,True
278,sec_app_mths_since_last_major_derog,Months since most recent 90-day or worse rati...,BrowseNotes,found,sec_app_mths_since_last_major_derog,int,True


## empty columns \with only null values will be: is_required=False

In [52]:
loans_columns =df_loans.columns

In [45]:
# חישוב סך ערכים שאינם null בכל עמודה
non_null_counts = df_loans.select([
    count(when(col(c).isNotNull(), c)).alias(c) for c in df_loans.columns
]).collect()[0].asDict()


In [47]:
empty_columns = [col for col, cnt in non_null_counts.items() if cnt == 0]

In [50]:
empty_columns

['id', 'member_id']

In [57]:
df_all['is_required'] = df_all.apply(
    lambda row: False if row['column_name'] not in loans_columns or row['column_name'] in empty_columns else row['is_required'],
    axis=1
)

In [65]:
df_all.to_csv("loans_mapping.csv", index=False, encoding='utf-8')


### Adding in the future:

df_merged["test_function"] = ""
df_merged["test_type"] = ""
df_merged["required"] = ""
df_merged["business_category"] = ""


In [61]:
def classify_column(description):
    d = str(description).lower()
    if 'income' in d or 'amount' in d or 'balance' in d:
        return 'funds'
    elif 'employer' in d or 'job' in d or 'title' in d:
        return 'customer details'
    elif 'status' in d or 'default' in d or 'delinquency' in d:
        return 'status'
    elif 'date' in d or 'month' in d:
        return 'dates'
    elif 'state' in d or 'zip' in d:
        return 'geographic location'
    else:
        'other'

df_merged['business_category'] = df_merged['Description'].apply(classify_column)
df_merged['business_Category']

df_merged.drop(columns=['business_Category'], inplace=True)


### Saving the file as main mapping file


df_merged.to_csv("test_mapping_master.csv", index=False, header=True)


df_merged.to_csv("test_mapping_master_clean.csv", index=False, encoding="utf-8", sep=",")


### mapping the testing functions 

### testing functions names:


['check_consistency', 'check_duplicates', 'check_foreign_key', 'check_nulls', 'count_records', 'dtype_consistency', 'duplicated_files_by_name_and_size', 'duplicated_files_current_path', 'file_timeliness', 'moving_average', 'validate_allowed_values', 'validate_date_format', 'validate_date_range', 'validate_numeric_range', 'validate_sum', 'validate_values_length']
